In [1]:
import pandas as pd
data = pd.read_csv("./data/participant_demo_clinical_all.csv")

In [2]:
def create_summary_table(df):
    from scipy import stats
    import pandas as pd

    # Preprocessing
    df.columns = df.columns.str.strip()  # Strip any whitespace from the column names

    # Selecting relevant columns
    selected_columns = ['GROUP', '2.AGE', '1. SEX'] + [col for col in df.columns if col.startswith(('LSAS', 'MOCI', 'BFNE', 'PSWQ', 'Handedness', 'FCV-19S'))]
    df_selected = df[selected_columns]

    # Renaming columns for better readability
    df_selected.rename(columns={'2.AGE': 'Age', '1. SEX': 'Sex'}, inplace=True)

    # Splitting data into experimental (EXP) and control (HC) groups
    exp_group = df_selected[df_selected['GROUP'] == 'EXP']
    hc_group = df_selected[df_selected['GROUP'] == 'HC']

    # Define a function to calculate mean (SD) and p-value
    def mean_sd(series):
        return f"{series.mean():.2f} ({series.std():.2f})"

    def calculate_pvalue(exp_series, hc_series):
        return stats.ttest_ind(exp_series, hc_series, nan_policy='omit').pvalue

    # Create a summary table
    summary_table = pd.DataFrame(columns=['Measure', 'EXP', 'HC', 'p-value'])

    for column in df_selected.columns[2:]:  # Exclude 'GROUP', 'Age', 'Sex' columns
        exp_mean_sd = mean_sd(exp_group[column])
        hc_mean_sd = mean_sd(hc_group[column])
        p_value = calculate_pvalue(exp_group[column], hc_group[column])
        
        summary_table = pd.concat([summary_table, pd.DataFrame([{'Measure': column, 'EXP': exp_mean_sd, 'HC': hc_mean_sd, 'p-value': p_value}])], ignore_index=True)

    # Adding Age and Sex columns separately
    summary_table = pd.concat([summary_table, pd.DataFrame([{
        'Measure': 'Age',
        'EXP': mean_sd(exp_group['Age']),
        'HC': mean_sd(hc_group['Age']),
        'p-value': calculate_pvalue(exp_group['Age'], hc_group['Age'])
    }])], ignore_index=True)

    # Sex should be handled as a percentage of females
    exp_female_pct = (exp_group['Sex'].value_counts(normalize=True).get(2, 0)) * 100
    hc_female_pct = (hc_group['Sex'].value_counts(normalize=True).get(2, 0)) * 100

    summary_table = pd.concat([summary_table, pd.DataFrame([{
        'Measure': 'Sex (Female %)',
        'EXP': f"{exp_female_pct:.2f}%",
        'HC': f"{hc_female_pct:.2f}%",
        'p-value': calculate_pvalue(exp_group['Sex'] == 2, hc_group['Sex'] == 2)
    }])], ignore_index=True)

    return summary_table

# Using the function to create the summary table
summary_df = create_summary_table(data)
summary_df

/tmp/ipykernel_528097/2808827663.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.rename(columns={'2.AGE': 'Age', '1. SEX': 'Sex'}, inplace=True)


,Measure,EXP,HC,p-value
0,Sex,1.46 (0.50),1.50 (0.50),5.733906e-01
1,LSAS_performance,42.88 (11.84),12.40 (8.75),3.784724e-39
2,LSAS_social_interaction,37.70 (11.85),10.53 (7.91),2.235185e-35
3,LSAS,80.58 (22.55),22.93 (16.01),2.237433e-39
4,MOCI,16.30 (5.67),20.96 (8.02),4.330711e-05
5,MOCI_checking,4.56 (2.26),6.36 (2.53),6.793133e-06
6,MOCI_cleaning,6.79 (3.39),8.30 (3.81),9.945855e-03
7,MOCI_doubting,3.31 (2.11),4.00 (1.74),3.019463e-02
8,MOCI_slowness,3.65 (1.81),5.39 (2.32),6.397710e-07
9,BFNE,50.85 (5.58),33.03 (8.29),4.988795e-34


In [11]:
summary_df.to_csv("./data/summary_all.csv")

In [10]:
print(data.groupby("GROUP").agg(['mean', 'std']))

from IPython.display import display, HTML

# DataFrame을 HTML로 변환하여 출력
display(HTML(data.groupby("GROUP").agg(['mean', 'std']).to_html()))

      Unnamed: 0                Exp No.                1. SEX            \
            mean        std        mean         std      mean       std   
GROUP                                                                     
EXP    88.788889  46.345671  261.755556  184.895434  1.455556  0.500811   
HC     72.635135  47.657906  211.040541  181.308595  1.500000  0.503413   

           2.AGE           3-2. YR_EDU            ... MOCI_slowness            \
            mean       std        mean       std  ...          mean       std   
GROUP                                             ...                           
EXP    28.866667  7.519563   14.812500  2.571185  ...      3.654762  1.813564   
HC     29.932432  7.192711   15.390411  2.415584  ...      5.385714  2.317500   

            BFNE                 PSWQ           Handedness(true)            \
            mean       std       mean       std             mean       std   
GROUP                                                         

/tmp/ipykernel_528097/4224565162.py:1: FutureWarning: ['HAID ID', 'fmri_code', 'Enrollment #'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  print(data.groupby("GROUP").agg(['mean', 'std']))
/tmp/ipykernel_528097/4224565162.py:6: FutureWarning: ['HAID ID', 'fmri_code', 'Enrollment #'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  display(HTML(data.groupby("GROUP").agg(['mean', 'std']).to_html()))
